# Data-Centric NLP 대회: 주제 분류 프로젝트

## Load Libraries

In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-11-04 12:45:40.641613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730691940.666196  449169 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730691940.673663  449169 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory 

## Set Hyperparameters

In [2]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [3]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [4]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
OUTPUT_DIR = os.path.join(BASE_DIR, '../output')

## Load Tokenizer and Model

In [5]:
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define Dataset

In [6]:
data = pd.read_csv(os.path.join(DATA_DIR, 'expanded_augmented.csv'))
dataset_train, dataset_valid = train_test_split(data, test_size=0.2, random_state=SEED)

In [7]:
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['augmented_text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

In [8]:
data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define Metric

In [10]:
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')


## Train Model

In [11]:
### for wandb setting
os.environ['WANDB_DISABLED'] = 'true'

In [12]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
trainer.train()

Step,Training Loss,Validation Loss,F1
100,1.453700,0.933214,0.711838
200,0.806800,0.629050,0.808794
300,0.528300,0.507877,0.843190
400,0.406200,0.455182,0.860810


TrainOutput(global_step=410, training_loss=0.7893469694184094, metrics={'train_runtime': 49.0503, 'train_samples_per_second': 266.42, 'train_steps_per_second': 8.359, 'total_flos': 167726438308200.0, 'train_loss': 0.7893469694184094, 'epoch': 2.0})

## Evaluate Model

In [15]:
dataset_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [16]:
from torch.utils.data import DataLoader
from datasets import Dataset as HF_Dataset

# 1. 데이터프레임을 Hugging Face Dataset으로 변환
hf_dataset = HF_Dataset.from_pandas(dataset_test[['text']])  # 'text' 컬럼만 사용

# 2. 전처리 함수 정의
def preprocess_function(examples):
    return tokenizer(examples['text'])

# 3. map을 이용해 전처리 함수 적용 (batched=True)
tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)

# Hugging Face Dataset을 PyTorch 호환 형식으로 변환
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# 4. DataLoader 생성
test_loader = DataLoader(tokenized_dataset, batch_size=600, collate_fn=data_collator)

# 5. 모델 평가
model.eval()
preds = []

for batch in tqdm(test_loader, desc="Evaluating"):
    inputs = {key: value.to(DEVICE) for key, value in batch.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        preds.extend(pred)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Evaluating: 100%|██████████| 50/50 [00:13<00:00,  3.83it/s]


In [17]:
# model.eval()
# preds = []

# for idx, sample in tqdm(dataset_test.iterrows(), total=len(dataset_test), desc="Evaluating"):
#     inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
#     with torch.no_grad():
#         logits = model(**inputs).logits
#         pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
#         preds.extend(pred)

In [18]:
dataset_test['target'] = preds
dataset_test.to_csv(os.path.join(BASE_DIR, 'output.csv'), index=False)